In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import metrics
from sklearn.metrics import precision_recall_fscore_support, accuracy_score


# Zadanie 1

In [2]:
(trainX, trainY), (testX, testY) = tf.keras.datasets.mnist.load_data()
trainX = tf.cast(trainX, tf.float32) / 255.
testX = tf.cast(testX, tf.float32) / 255.


### Sequential model

In [3]:
sequential_model = keras.Sequential(
    [
        layers.Flatten(input_shape=(28, 28)),
        layers.Dense(128, activation="relu"),
        layers.Dense(128, activation="relu"),
        layers.Dense(10, activation="softmax"),
    ]
)
sequential_model.summary()
sequential_model.load_weights("model_w.h5")
sequential_model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=[metrics.CategoricalAccuracy(), metrics.Precision(), metrics.Recall()])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 10)                1290      
                                                                 
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


#### model.evaluate()

In [4]:
(loss, accuracy_sequential_eval, precision_sequential_eval, recall_sequential_eval) = sequential_model.evaluate(testX, tf.one_hot(testY, depth=10), verbose=1)
print("Accuracy: ", accuracy_sequential_eval)
print("Precision: ", precision_sequential_eval)
print("Recall: ", recall_sequential_eval)

313/313 [==============================] - 0s 628us/step - loss: 0.0768 - categorical_accuracy: 0.9778 - precision: 0.9789 - recall: 0.9762
Accuracy:  0.9778000116348267
Precision:  0.978941023349762
Recall:  0.9761999845504761


#### scikit-learn metrics (prediction)

In [5]:
preds_sequential = sequential_model.predict(testX)
accuracy_sequential = accuracy_score(testY, preds_sequential.argmax(axis=1))
precision_sequential, recall_sequential, _, _ = precision_recall_fscore_support(
    testY, preds_sequential.argmax(axis=1), average='macro')
print("Accuracy: ", accuracy_sequential)
print("Precision: ", precision_sequential)
print("Recall: ", recall_sequential)


313/313 [==============================] - 0s 532us/step
Accuracy:  0.9778
Precision:  0.9778581534433096
Recall:  0.9775777296255154


### Functional model

In [6]:
inputs = layers.Input(shape=(28, 28))
flatten = layers.Flatten()(inputs)
dense1 = layers.Dense(128, activation="relu")(flatten)
dense2 = layers.Dense(128, activation="relu")(dense1)
outputs = layers.Dense(10, activation="softmax")(dense2)
functional_model = keras.Model(inputs=inputs, outputs=outputs)
functional_model.summary()
functional_model.load_weights("model_w.h5")
functional_model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=[metrics.CategoricalAccuracy(), metrics.Precision(), metrics.Recall()]
)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_3 (Dense)             (None, 128)               100480    
                                                                 
 dense_4 (Dense)             (None, 128)               16512     
                                                                 
 dense_5 (Dense)             (None, 10)                1290      
                                                                 
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


#### model.evaluate()

In [10]:
(loss, accuracy_functional_eval, precision_functional_eval, recall_functional_eval) = functional_model.evaluate(testX, tf.one_hot(testY, depth=10), verbose=1)
print("Accuracy: ", accuracy_functional_eval)
print("Precision: ", precision_functional_eval)
print("Recall: ", recall_functional_eval)

313/313 [==============================] - 0s 591us/step - loss: 0.0768 - categorical_accuracy: 0.9778 - precision_1: 0.9789 - recall_1: 0.9762
Accuracy:  0.9778000116348267
Precision:  0.978941023349762
Recall:  0.9761999845504761


#### scikit-learn metrics (prediction)

In [8]:
preds_functional = functional_model.predict(testX)
accuracy_functional = accuracy_score(testY, preds_functional.argmax(axis=1))
precision_functional, recall_functional, _, _ = precision_recall_fscore_support(
    testY, preds_functional.argmax(axis=1), average='macro')
print("Accuracy: ", accuracy_functional)
print("Precision: ", precision_functional)
print("Recall: ", recall_functional)


313/313 [==============================] - 0s 500us/step
Accuracy:  0.9778
Precision:  0.9778581534433096
Recall:  0.9775777296255154


# Zadanie 2

In [9]:
scalar = layers.Input(shape=(), dtype=tf.float32)
batch = layers.Input(shape=(28, 28), dtype=tf.float32)
preds_raw = sequential_model(batch)
preds_sum = layers.Add()([preds_raw, scalar])
model = keras.Model(inputs=[scalar, batch], outputs=[preds_raw, preds_sum])
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",
              metrics=[metrics.SparseCategoricalAccuracy()])
raw_preds, sum_preds = model.predict(
    [tf.constant([5.0]*5, dtype=tf.float32), testX[:5]])
print("Raw predictions: ", raw_preds)
print("Sum predictions: ", sum_preds)
print("Raw predictions (indices): ", tf.argmax(raw_preds, axis=1))
print("Sum predictions (indices): ", tf.argmax(sum_preds, axis=1))
print("Test Y values: ", testY[:5])


1/1 [==============================] - 0s 29ms/step
Raw predictions:  [[2.9017097e-10 9.9630810e-08 9.1427519e-06 8.2642044e-05 2.4783178e-09
  1.3760470e-08 2.5339904e-12 9.9990022e-01 8.1127148e-08 7.8429593e-06]
 [2.0258680e-11 4.6203073e-07 9.9999869e-01 6.8475242e-07 2.3875413e-16
  2.0451862e-08 9.4727164e-11 1.1061840e-08 9.8418937e-08 4.8029636e-12]
 [8.7542776e-07 9.9791926e-01 8.5040723e-05 5.1883439e-06 8.6449640e-05
  2.7820242e-06 5.2900217e-05 1.4191855e-03 4.1309578e-04 1.5299325e-05]
 [9.9987257e-01 1.6904997e-07 9.9806144e-05 1.1384545e-07 5.2204527e-08
  3.0985743e-07 4.4615067e-06 8.5699612e-06 2.8328417e-09 1.3971470e-05]
 [1.1878739e-08 2.2973106e-09 2.3022245e-08 1.4703636e-08 9.9937505e-01
  2.6422565e-06 1.1956548e-08 3.0697925e-06 2.3329465e-06 6.1690516e-04]]
Sum predictions:  [[5.        5.        5.000009  5.0000825 5.        5.        5.
  5.9999003 5.        5.0000076]
 [5.        5.0000005 5.9999986 5.0000005 5.        5.        5.
  5.        5.        5